In [ ]:
! pip install lightning --quiet

In [ ]:
import torch
import torch.nn as nn

from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from torch.optim import AdamW

import lightning as L

import plotly.express as px

In [ ]:
time_series = torch.sin(torch.arange(0, 100, 0.1))

In [ ]:
fig = px.line(time_series, template='plotly_dark', markers=True)
fig.show()

In [ ]:
class MyDataset(Dataset):
    def __init__(self, time_series, window_size: int = 20):
        self.time_series = time_series
        self.window_size = window_size
    
    def __len__(self):
        return len(time_series) - self.window_size

    def __getitem__(self, idx):
        return self.time_series[idx:idx+self.window_size], self.time_series[idx+self.window_size]

In [ ]:
class TSDataModuleNoShuffle(L.LightningDataModule):
    def __init__(self, time_series, batch_size: int = 32):
        super().__init__()
        self.time_series = time_series
        self.batch_size = batch_size

    def setup(self, stage: str):
        self.test_dataset = MyDataset(self.time_series)

    def train_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size, drop_last=True, shuffle=False)

In [ ]:
data_module = TSDataModuleNoShuffle(time_series=time_series)

In [ ]:
class MyRnn(L.LightningModule):
    def __init__(self, bidirectional: bool = False):
        super().__init__()
        self.loss = nn.MSELoss()
        self.bidirectional = bidirectional

        self.rnn = nn.RNN(input_size=1, hidden_size=16, 
                          bidirectional=bidirectional, batch_first=True)
        if bidirectional:
            self.linear = nn.Linear(32, 1)
        else:
            self.linear = nn.Linear(16, 1)
        
    
    def forward(self, input):
        output, _ = self.rnn(input)
        output = self.linear(output[:, -1, :])
        return output
    
    def configure_optimizers(self):
        optimizer = AdamW(self.parameters())
        return optimizer
    
    def training_step(self, batch, batch_idx):
        X, y = batch
        output = self.forward(X.unsqueeze(2))
        loss = self.loss(output.flatten(), y)
        self.log('train_loss', loss, prog_bar=True)
        return {'loss': loss}
    


In [ ]:
model = MyRnn()

In [ ]:
trainer = L.Trainer(max_epochs=20)

INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(model, data_module)

INFO: 
  | Name   | Type    | Params
-----------------------------------
0 | loss   | MSELoss | 0     
1 | rnn    | RNN     | 304   
2 | linear | Linear  | 17    
-----------------------------------
321       Trainable params
0         Non-trainable params
321       Total params
0.001     Total estimated model params size (MB)
INFO:lightning.pytorch.callbacks.model_summary:
  | Name   | Type    | Params
-----------------------------------
0 | loss   | MSELoss | 0     
1 | rnn    | RNN     | 304   
2 | linear | Linear  | 17    
-----------------------------------
321       Trainable params
0         Non-trainable params
321       Total params
0.001     Total estimated model params size (MB)
/usr/local/lib/python3.10/dist-packages/lightning/pytorch/loops/fit_loop.py:280: PossibleUserWarning:

The number of training batches (30) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.



Training: 0it [00:00, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=20` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=20` reached.


In [ ]:
X_test = torch.sin(torch.arange(1000, 1002, 0.1))

In [ ]:
model(X_test.unsqueeze(0).unsqueeze(2))

tensor([[0.1555]], grad_fn=<AddmmBackward0>)

In [ ]:
torch.sin(torch.tensor(1002))

tensor(0.1673)

In [ ]:
model = MyRnn(bidirectional=True)

In [ ]:
trainer = L.Trainer(max_epochs=20)

INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(model, data_module)

INFO: 
  | Name   | Type    | Params
-----------------------------------
0 | loss   | MSELoss | 0     
1 | rnn    | RNN     | 608   
2 | linear | Linear  | 33    
-----------------------------------
641       Trainable params
0         Non-trainable params
641       Total params
0.003     Total estimated model params size (MB)
INFO:lightning.pytorch.callbacks.model_summary:
  | Name   | Type    | Params
-----------------------------------
0 | loss   | MSELoss | 0     
1 | rnn    | RNN     | 608   
2 | linear | Linear  | 33    
-----------------------------------
641       Trainable params
0         Non-trainable params
641       Total params
0.003     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=20` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=20` reached.


In [ ]:
model(X_test.unsqueeze(0).unsqueeze(2))

tensor([[0.1544]], grad_fn=<AddmmBackward0>)

In [ ]:
torch.sin(torch.tensor(1002))

tensor(0.1673)

In [ ]:
class TSDataModuleShuffle(L.LightningDataModule):
    def __init__(self, time_series, batch_size: int = 32):
        super().__init__()
        self.time_series = time_series
        self.batch_size = batch_size

    def setup(self, stage: str):
        self.test_dataset = MyDataset(self.time_series)

    def train_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size, drop_last=True, shuffle=True)

In [ ]:
data_module = TSDataModuleShuffle(time_series=time_series)

In [ ]:
model = MyRnn()

In [ ]:
trainer = L.Trainer(max_epochs=20)

INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(model, data_module)

INFO: 
  | Name   | Type    | Params
-----------------------------------
0 | loss   | MSELoss | 0     
1 | rnn    | RNN     | 304   
2 | linear | Linear  | 17    
-----------------------------------
321       Trainable params
0         Non-trainable params
321       Total params
0.001     Total estimated model params size (MB)
INFO:lightning.pytorch.callbacks.model_summary:
  | Name   | Type    | Params
-----------------------------------
0 | loss   | MSELoss | 0     
1 | rnn    | RNN     | 304   
2 | linear | Linear  | 17    
-----------------------------------
321       Trainable params
0         Non-trainable params
321       Total params
0.001     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=20` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=20` reached.


In [ ]:
X_test = torch.sin(torch.arange(1000, 1002, 0.1))

In [ ]:
model(X_test.unsqueeze(0).unsqueeze(2))

tensor([[0.1739]], grad_fn=<AddmmBackward0>)

In [ ]:
torch.sin(torch.tensor(1002))

tensor(0.1673)

In [ ]:
model = MyRnn(bidirectional=True)

In [ ]:
trainer = L.Trainer(max_epochs=20)

INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(model, data_module)

INFO: 
  | Name   | Type    | Params
-----------------------------------
0 | loss   | MSELoss | 0     
1 | rnn    | RNN     | 608   
2 | linear | Linear  | 33    
-----------------------------------
641       Trainable params
0         Non-trainable params
641       Total params
0.003     Total estimated model params size (MB)
INFO:lightning.pytorch.callbacks.model_summary:
  | Name   | Type    | Params
-----------------------------------
0 | loss   | MSELoss | 0     
1 | rnn    | RNN     | 608   
2 | linear | Linear  | 33    
-----------------------------------
641       Trainable params
0         Non-trainable params
641       Total params
0.003     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=20` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=20` reached.


In [ ]:
model(X_test.unsqueeze(0).unsqueeze(2))

tensor([[0.1603]], grad_fn=<AddmmBackward0>)

In [ ]:
torch.sin(torch.tensor(1002))

tensor(0.1673)